In [1]:
import matplotlib.pyplot as plt
from rdkit import Chem
import numpy as np
import pandas as pd
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Linear
from torch.nn import BatchNorm1d
import torch.optim as optim
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_add_pool
from torch_geometric.data import DataLoader, Data
import copy
import time
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, balanced_accuracy_score
from tqdm import tqdm
import openpyxl
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr"

c:\Users\trifo\anaconda3\envs\GCNenv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
parser = argparse.ArgumentParser()
args = parser.parse_args("")
args.seed = 123

np.random.seed(args.seed)
torch.manual_seed(args.seed)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
#https://pytorch-geometric.readthedocs.io/en/latest/notes/introduction.html for reference

#two types of onehot encoding
def onehot_encoding_unk(x, allowable_set):
    #maps input not in the allowable set to the last element.
    if x not in allowable_set:
        x = allowable_set[-1]
    return list(map(lambda s: x == s, allowable_set))
def onehot_encoding(x, allowable_set):
    #raises exception if input not in allowable set
    if x not in allowable_set:
        raise Exception("input {0} not in allowable set {1}:".format(x, allowable_set))
    return list(map(lambda s: x == s, allowable_set))


#returns 1D array of node (atom) features 
#function will be used later to return a matrix with shape of [num_atoms, num_atom_features] per compound/chemical
atom_types = ['C','N','O','S','F','Si','P','Cl','Br','Mg','Na','Ca','Fe','As','Al','I',
              'B','V','K','Tl', 'Yb','Sb','Sn','Ag','Pd','Co','Se','Ti','Zn','H','Li',
              'Ge','Cu','Au','Ni','Cd','In','Mn','Zr','Cr','Pt','Hg','Pb','Unknown']
atom_degrees = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
atom_implicit_valences = [0, 1, 2, 3, 4, 5, 6]
atom_hybridizations = [Chem.rdchem.HybridizationType.SP, Chem.rdchem.HybridizationType.SP2, Chem.rdchem.HybridizationType.SP3, Chem.rdchem.HybridizationType.SP3D, Chem.rdchem.HybridizationType.SP3D2]
atom_num_Hs = [0, 1, 2, 3, 4]
def atom_featurization(atom):
    #featurization of each atom
    atom_feats = np.array(onehot_encoding_unk(atom.GetSymbol(), atom_types) 
    + onehot_encoding(atom.GetDegree(), atom_degrees) 
    + onehot_encoding_unk(atom.GetImplicitValence(), atom_implicit_valences)
    + [atom.GetFormalCharge()] 
    + [atom.GetNumRadicalElectrons()] 
    + onehot_encoding_unk(atom.GetHybridization(), atom_hybridizations) 
    + [atom.GetIsAromatic()] 
    + onehot_encoding_unk(atom.GetTotalNumHs(), atom_num_Hs))
    return atom_feats


#returns 1D array of edge (bond) features
#function will be used later to return matrix of edge (bond) features with shape of [num_bonds, num_bond_features]
bond_types = [Chem.rdchem.BondType.SINGLE, Chem.rdchem.BondType.DOUBLE, Chem.rdchem.BondType.TRIPLE, Chem.rdchem.BondType.AROMATIC]
def bond_featurization(bond):
    #featurization of each bond
    bond_type = bond.GetBondType()
    bond_feats = np.array([bond_type == Chem.rdchem.BondType.SINGLE, bond_type == Chem.rdchem.BondType.DOUBLE, bond_type == Chem.rdchem.BondType.TRIPLE, bond_type == Chem.rdchem.BondType.AROMATIC] 
    + [bond.GetIsConjugated()] 
    + [bond.IsInRing()])
    return bond_feats


#returns list of index tuples (bond indexes) that accounts for both directions of each bond
def bond_pairs_between_atoms(mol):                                   
    bonds = mol.GetBonds()
    bond_index_tuples = []
    for bond in bonds:
        bond_index_tuples.append([bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()]) #forward direction e.g. [0,1]
        bond_index_tuples.append([bond.GetEndAtomIdx(), bond.GetBeginAtomIdx()]) #backward direction [1,0]
    return bond_index_tuples


#returns a pyg Data object from MOL objects using the functions above
def mol_to_graph(mol):
    atoms = mol.GetAtoms()
    bonds = mol.GetBonds()
    node_features= [atom_featurization(atom) for atom in atoms]

    #Applies bond_featurization function twice to make it the same dimensions as the edge_index matrix since both directions of the bond is accounted for
    edge_features = [bond_featurization(bond) for bond in bonds]
    for bond in bonds:
        edge_features.append(bond_featurization(bond))
    #
    
    edge_index = bond_pairs_between_atoms(mol)
    #pyg Data object construction
    data = Data(x=torch.tensor(node_features, dtype=torch.float),
                edge_index=torch.tensor(edge_index, dtype=torch.long).t().contiguous(),
                edge_attr=torch.tensor(edge_features, dtype=torch.float))
    return data

#returns dict with mol object as the key and the value being the class (nonreadibly biodegradeable VS readibly biodegradeable).
def MOL_class_dict_from_SMILES(dataframe):
    mols_class_dict = {}
    for i in range(dataframe.shape[0]):
        mols_class_dict[Chem.MolFromSmiles(dataframe['SMILES'].iloc[i])] = dataframe['Class'].iloc[i]
    return mols_class_dict

#returns list of all pyg Data objects for each SMILES/MOL
#Each Data object contains the graph information (node features, edge features, and edge index) as well as the ground truth/classification (NRB vs RB)
def make_all_graphs(mols_class_dict):
    X = [mol_to_graph(m) for m in mols_class_dict.keys()]
    #Adds the classification/ground truth field, object_variable_name.y, for each pyg Data object
    for i, dataobject in enumerate(X):
        dataobject.y = torch.tensor([list(mols_class_dict.values())[i]], dtype=torch.long) 
    #
    return X

In [4]:
#implementing architecture of the model: 4-layer GCN and 3-layer MLP combined
class GCNlayer(nn.Module):
    
    def __init__(self, n_features, conv_dim1, conv_dim2, conv_dim3, concat_dim, dropout):
        super(GCNlayer, self).__init__()
        self.n_features = n_features
        self.conv_dim1 = conv_dim1
        self.conv_dim2 = conv_dim2
        self.conv_dim3 = conv_dim3
        self.concat_dim =  concat_dim
        self.dropout = dropout
        
        self.conv1 = GCNConv(self.n_features, self.conv_dim1)
        self.bn1 = BatchNorm1d(self.conv_dim1)
        self.conv2 = GCNConv(self.conv_dim1, self.conv_dim2)
        self.bn2 = BatchNorm1d(self.conv_dim2)
        self.conv3 = GCNConv(self.conv_dim2, self.conv_dim3)
        self.bn3 = BatchNorm1d(self.conv_dim3)
        self.conv4 = GCNConv(self.conv_dim3, self.concat_dim)
        self.bn4 = BatchNorm1d(self.concat_dim)
        
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = self.bn1(x)
        x = F.relu(self.conv2(x, edge_index))
        x = self.bn2(x)
        x = F.relu(self.conv3(x, edge_index))
        x = self.bn3(x)
        x = F.relu(self.conv4(x, edge_index))
        x = self.bn4(x)
        x = global_add_pool(x, data.batch)
        x = F.dropout(x, p=self.dropout, training=self.training)
        return x
    
class FClayer(nn.Module): 
    
    def __init__(self, concat_dim, pred_dim1, pred_dim2, out_dim, dropout):
        super(FClayer, self).__init__()
        self.concat_dim = concat_dim
        self.pred_dim1 = pred_dim1
        self.pred_dim2 = pred_dim2
        self.out_dim = out_dim
        self.dropout = dropout

        self.fc1 = Linear(self.concat_dim, self.pred_dim1)
        self.bn1 = BatchNorm1d(self.pred_dim1)
        self.fc2 = Linear(self.pred_dim1, self.pred_dim2)
        self.fc3 = Linear(self.pred_dim2, self.out_dim)
    
    def forward(self, data):
        x = F.relu(self.fc1(data))
        x = self.bn1(x)
        x = F.relu(self.fc2(x))
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.fc3(x)
        return x
    
class Net(nn.Module):
    def __init__(self, args):
        super(Net, self).__init__()
        self.conv = GCNlayer(args.n_features, 
                              args.conv_dim1, 
                              args.conv_dim2, 
                              args.conv_dim3, 
                              args.concat_dim, 
                              args.dropout)

        self.fc = FClayer(args.concat_dim, 
                          args.pred_dim1, 
                          args.pred_dim2, 
                          args.out_dim, 
                          args.dropout) 
        
    def forward(self, data):
        x = self.conv(data)
        x = self.fc(x)
        x = F.log_softmax(x, dim=1)
        return x

In [5]:
#save model state for transfer learning later, called in experiment function below
def save_checkpoint(epoch, model, optimizer, scheduler, filename):
    state = {
        'Epoch': epoch,
        'State_dict': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'scheduler': scheduler.state_dict()
    }
    torch.save(state, filename)

#training model on training set, called in experiment function below
def train(model, device, optimizer, train_loader, criterion, args):
    train_correct = 0
    train_total = 0
    epoch_train_loss = 0
    for i, data in enumerate(train_loader):
        data = data.to(device)
        labels = data.y.to(device)
        optimizer.zero_grad()
        outputs = model(data)
        outputs.require_grad = False
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()
        loss = criterion(outputs, labels)
        epoch_train_loss += loss.item()
        loss.backward()
        optimizer.step()
    epoch_train_loss /= len(train_loader)
    train_acc =  100 * train_correct / train_total
    return model, epoch_train_loss, train_acc

#evaluating performance metrics of trained model on the test set, called in experiment function below
def test(model, device, test_loader, args): 
    model.eval()
    classes = ('RB', 'NRB')
    class_correct = list(0. for i in range(len(classes)))
    class_total = list(0. for i in range(len(classes)))
    correct = 0
    total = 0
    nb_classes = len(classes)
    confusion_matrix = torch.zeros(nb_classes, nb_classes)
    test_hist = {"test_acc":[]}
    y_score =[]
    y_test =[]
    data_total = []
    pred_data_total = []
    with torch.no_grad():
        for i, data in enumerate(test_loader):
            data = data.to(device)
            labels = data.y.to(device)
            data_total += data.y.tolist()
            outputs = model(data)
            pred_data_total += outputs.view(-1).tolist()
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
            y_score.append(outputs.cpu().numpy()) 
            y_test.append(labels.cpu().numpy()) 
            for t, p in zip(labels.view(-1), predicted.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1
            for i in range(labels.shape[0]):
                if(labels.shape[0] == 1):
                    label = labels[i]
                    class_correct[label] += c.item()
                    class_total[label] += 1
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
                    test_hist["test_acc"].append((predicted == labels).sum().item())
                else:  
                    label = labels[i]
                    class_correct[label] += c[i].item()
                    class_total[label] += 1
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
                    test_hist["test_acc"].append((predicted == labels).sum().item())

    data_total = list(labels.view(-1).cpu().numpy()) #true class
    pred_data_total = list(predicted.view(-1).cpu().numpy()) #model predicted class, create cm using this
    y_pred_list = [a.squeeze().tolist() for a in y_score][0]
    conf = confusion_matrix.tolist()
    total_acc = 100 * correct / total
    rb = -1
    nrb = -1
    if class_total[0] != 0:
        rb = 100 * class_correct[0] / class_total[0]
    if class_total[1] != 0:
        nrb = 100 * class_correct[1] / class_total[1]
    miscore = f1_score(data_total, pred_data_total, average='micro')
    mascore = f1_score(data_total, pred_data_total, average='macro')
    ba = balanced_accuracy_score(data_total, pred_data_total)
    er = (confusion_matrix[0,1]+confusion_matrix[1,0])/sum(sum(confusion_matrix))
    return conf, total_acc, rb, nrb, miscore, mascore, data_total, pred_data_total, y_pred_list, ba, er

# Overall function that takes the defined model architecture, training set, testing set, device to use, and hyperparameters. Utilizes both functions above.
def experiment(model, train_loader, test_loader, device, args, trainmodel = True, savemodel = True): 
    time_start = time.time()
    
    optimizer = optim.Adam(model.parameters(),lr=args.lr)
    criterion = nn.CrossEntropyLoss()
    scheduler = optim.lr_scheduler.StepLR(optimizer,
                                          step_size=args.step_size,
                                          gamma=args.gamma)
    #training here
    list_train_loss = list()
    list_train_acc = list()
    if trainmodel:
        #print('[Train]')
        for epoch in range(args.epoch):
            scheduler.step()
            #print('- Epoch :', epoch+1)
            model, train_loss, train_acc = train(model, device, optimizer, train_loader, criterion, args)
            list_train_loss.append(train_loss)
            list_train_acc.append(train_acc)
            #print('Loss : %.4f' % list_train_loss[-1], '| Accuracy : %.4f' % list_train_acc[-1])
    
    #testing here
    conf, total_acc, RB, NRB, miscore, mascore, data_total, pred_data_total, y_pred_list, ba, er = test(model, device, test_loader, args)
    
    time_end = time.time()
    time_required = time_end - time_start
    
    args.list_train_loss = list_train_loss
    args.list_train_acc = list_train_acc
    args.data_total = data_total
    args.pred_data_total = pred_data_total
    args.conf = conf
    args.total_acc = total_acc
    args.RB = RB
    args.NRB = NRB
    args.miscore = miscore
    args.mascore = mascore
    args.time_required = time_required
    args.y_pred_list = y_pred_list
    args.ba = ba
    args.er = er
    
    #saving model state here
    if savemodel:
        save_checkpoint(epoch, model, optimizer, scheduler, 'model.pt')

    return args

In [6]:
#Model application on original aggreated dataset (~3k)
args.epoch = 400
args.lr = 0.0001
args.optim = 'Adam'
args.step_size = 10
args.gamma = 0.9
args.dropout = 0.1
args.n_features = 75
dim = 512
args.conv_dim1 = dim
args.conv_dim2 = dim
args.conv_dim3 = dim
args.concat_dim = dim
args.pred_dim1 = dim
args.pred_dim2 = dim
args.out_dim = 2

BA = []
Sn = []
Sp = []
ER = []

t=0.2
r=42

np.random.seed(args.seed)
torch.manual_seed(args.seed)

df = pd.read_csv('C:/Users/trifo/Argonne SULI Project/src/Datasets\RB_vs_NRB_Compounds_(Tentative).csv', low_memory=False)
df = pd.concat([df['SMILES'], df['Class']], axis=1)

X_train, X_test = train_test_split(df, test_size=t, shuffle=True, stratify=df['Class'], random_state=r)
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

train_mols = MOL_class_dict_from_SMILES(X_train)
test_mols = MOL_class_dict_from_SMILES(X_test)

train_X = make_all_graphs(train_mols)
test_X = make_all_graphs(test_mols)

model = Net(args)
model = model.to(device)

train_loader = DataLoader(train_X, batch_size=len(train_X), shuffle=True, drop_last=True)
test_loader = DataLoader(test_X, batch_size=len(test_X), shuffle=False, drop_last=True)

dict_result = dict()
args.exp_name = 'Test'
result = vars(experiment(model, train_loader, test_loader, device, args, trainmodel = True, savemodel = True))
dict_result[args.exp_name] = copy.deepcopy(result)
torch.cuda.empty_cache()

res_df = pd.DataFrame(dict_result).transpose()
c = res_df['conf'].iloc[0]
confusion_matrix = np.array([c[0], c[1]], dtype=float)
cm = confusion_matrix
sensitivity1 = cm[1,1]/(cm[1,1]+cm[1,0])
specificity1 = cm[0,0]/(cm[0,0]+cm[0,1])
er = (cm[0,1]+cm[1,0])/sum(sum(cm))
BA.append(res_df['ba'].iloc[0])
Sn.append(sensitivity1)
Sp.append(specificity1)
ER.append(er)
results_df = pd.DataFrame({'GCN_BA':BA, 'GCN_Sn':Sn, 'GCN_Sp':Sp, 'GCN_ER':ER})
results_df.to_csv('GCN_metrics.csv', index=False)
results_df

c:\Users\trifo\anaconda3\envs\GCNenv\lib\site-packages\ipykernel_launcher.py:74: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:210.)
c:\Users\trifo\anaconda3\envs\GCNenv\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
c:\Users\trifo\anaconda3\envs\GCNenv\lib\site-packages\torch\optim\lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/op

,GCN_BA,GCN_Sn,GCN_Sp,GCN_ER
0,0.852125,0.83105,0.873199,0.14311


In [32]:
#testing perfomance of model on new domain/dataset of polymers, NO TRAINING done here
#USING FRESHWATER DATASET
tl_BA = []
tl_Sn = []
tl_Sp = []
tl_ER = []

tl_df = pd.read_excel('C:/Users/trifo/Argonne SULI Project/src/GCN_RB_NRB_Model/TL_Freshwater__Dataset.xlsx')
tl_test = pd.concat([tl_df['SMILES'], tl_df['Class']], axis=1)
tl_test_mols = MOL_class_dict_from_SMILES(tl_test)
tl_test_X = make_all_graphs(tl_test_mols)

tl_train_loader = DataLoader([])
tl_test_loader = DataLoader(tl_test_X, batch_size=len(tl_test_X), shuffle=False, drop_last = True)

tl_dict_result = dict()
args.exp_name = 'tl_test'
tl_result = vars(experiment(model, tl_train_loader, tl_test_loader, device, args, trainmodel = False, savemodel = False))
tl_dict_result[args.exp_name] = copy.deepcopy(tl_result)
torch.cuda.empty_cache()

tl_res_df = pd.DataFrame(tl_dict_result).transpose()
tl_conf = tl_res_df['conf'].iloc[0]
tl_confusion_matrix = np.array([tl_conf[0], tl_conf[1]], dtype=float)
tlcm = tl_confusion_matrix
tl_sensitivity1 = tlcm[1,1]/(tlcm[1,1]+tlcm[1,0])
tl_specificity1 = tlcm[0,0]/(tlcm[0,0]+tlcm[0,1])
tl_er = (tlcm[0,1]+tlcm[1,0])/sum(sum(tlcm))
tl_BA.append(tl_res_df['ba'].iloc[0])
tl_Sn.append(tl_sensitivity1)
tl_Sp.append(tl_specificity1)
tl_ER.append(tl_er)

tl_results_df = pd.DataFrame({'GCN_tl_BA':tl_BA, 'GCN_tl_Sn':tl_Sn, 'GCN_tl_Sp':tl_Sp, 'GCN_tl_ER':tl_ER})
tl_results_df.to_csv('GCN_tl_metrics.csv', index=False)
print(tlcm)
display(tl_results_df)
#without training model predicts all as RB (1), so it predicts 2 incorrectly since PBS and PBAdip are NRB (0)

[[0. 2.]
 [0. 4.]]


c:\Users\trifo\anaconda3\envs\GCNenv\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


,GCN_tl_BA,GCN_tl_Sn,GCN_tl_Sp,GCN_tl_ER
0,0.5,1.0,0.0,0.333333


In [33]:
#testing perfomance of model on new domain/dataset of polymers, NO TRAINING done here
#USING SEAWATER(Marine) Dataset
tl_BA = []
tl_Sn = []
tl_Sp = []
tl_ER = []

tl_df = pd.read_excel('C:/Users/trifo/Argonne SULI Project/src/GCN_RB_NRB_Model/TL_Seawater(Marine)_Dataset.xlsx')
tl_test = pd.concat([tl_df['SMILES'], tl_df['Class']], axis=1)
tl_test_mols = MOL_class_dict_from_SMILES(tl_test)
tl_test_X = make_all_graphs(tl_test_mols)

tl_train_loader = DataLoader([])
tl_test_loader = DataLoader(tl_test_X, batch_size=len(tl_test_X), shuffle=False, drop_last = True)

tl_dict_result = dict()
args.exp_name = 'tl_test'
tl_result = vars(experiment(model, tl_train_loader, tl_test_loader, device, args, trainmodel = False, savemodel = False))
tl_dict_result[args.exp_name] = copy.deepcopy(tl_result)
torch.cuda.empty_cache()

tl_res_df = pd.DataFrame(tl_dict_result).transpose()
tl_conf = tl_res_df['conf'].iloc[0]
tl_confusion_matrix = np.array([tl_conf[0], tl_conf[1]], dtype=float)
tlcm = tl_confusion_matrix
tl_sensitivity1 = tlcm[1,1]/(tlcm[1,1]+tlcm[1,0])
tl_specificity1 = tlcm[0,0]/(tlcm[0,0]+tlcm[0,1])
tl_er = (tlcm[0,1]+tlcm[1,0])/sum(sum(tlcm))
tl_BA.append(tl_res_df['ba'].iloc[0])
tl_Sn.append(tl_sensitivity1)
tl_Sp.append(tl_specificity1)
tl_ER.append(tl_er)

tl_results_df = pd.DataFrame({'GCN_tl_BA':tl_BA, 'GCN_tl_Sn':tl_Sn, 'GCN_tl_Sp':tl_Sp, 'GCN_tl_ER':tl_ER})
tl_results_df.to_csv('GCN_tl_metrics.csv', index=False)
print(tlcm)
display(tl_results_df)
#predicting all of the values as 1, so 9 polymers that are 0 (NRB) are being predicted incorrectly

[[0. 9.]
 [0. 6.]]


c:\Users\trifo\anaconda3\envs\GCNenv\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


,GCN_tl_BA,GCN_tl_Sn,GCN_tl_Sp,GCN_tl_ER
0,0.5,1.0,0.0,0.6


In [44]:
#transfer learning through LOO (leave one out) validation for FRESHWATER

#altering experiment function for this portion
def kf_experiment(model, optimizer, scheduler, train_loader, test_loader, device, args, trainmodel = True, savemodel = False): 
    time_start = time.time()
    
    criterion = nn.CrossEntropyLoss()

    #training here
    list_train_loss = list()
    list_train_acc = list()
    if trainmodel:
        #print('[Train]')
        for epoch in range(args.epoch):
            scheduler.step()
            #print('- Epoch :', epoch+1)
            model, train_loss, train_acc = train(model, device, optimizer, train_loader, criterion, args)
            list_train_loss.append(train_loss)
            list_train_acc.append(train_acc)
            #print('Loss : %.4f' % list_train_loss[-1], '| Accuracy : %.4f' % list_train_acc[-1])
    
    #testing here
    conf, total_acc, RB, NRB, miscore, mascore, data_total, pred_data_total, y_pred_list, ba, er = test(model, device, test_loader, args)
    
    time_end = time.time()
    time_required = time_end - time_start
    
    args.list_train_loss = list_train_loss
    args.list_train_acc = list_train_acc
    args.data_total = data_total
    args.pred_data_total = pred_data_total
    args.conf = conf
    args.total_acc = total_acc
    args.RB = RB
    args.NRB = NRB
    args.miscore = miscore
    args.mascore = mascore
    args.time_required = time_required
    args.y_pred_list = y_pred_list
    args.ba = ba
    args.er = er
    
    #saving model state here
    if savemodel:
        save_checkpoint(epoch, model, optimizer, 'model.pt')

    return args

#ai for pfas code, transferlearning notebook, benchmarks bayesian gpyops

####### Model application
args.epoch = 1000
args.lr = 0.0001
args.optim = 'Adam'
args.step_size = 10
args.gamma = 0.9
args.dropout = 0.1
args.n_features = 75
dim = 512
args.conv_dim1 = dim
args.conv_dim2 = dim
args.conv_dim3 = dim
args.concat_dim = dim
args.pred_dim1 = dim
args.pred_dim2 = dim
args.out_dim = 2

kf_df = pd.read_excel('C:/Users/trifo/Argonne SULI Project/src/GCN_RB_NRB_Model/TL_Freshwater__Dataset.xlsx')
kf_df = pd.concat([kf_df['SMILES'], kf_df['Class']], axis=1)
kfold = KFold(n_splits=len(kf_df))

kf_BA = []
kf_Sn = []
kf_Sp = []
kf_ER = []

CM_list = []

validation = []

for fold, (train_index, test_index) in enumerate(kfold.split(kf_df)):
    print('Fold', fold+1, '...')
    kf_train = kf_df.iloc[train_index]
    kf_test = kf_df.iloc[test_index]
    kf_train = kf_train.reset_index(drop=True)
    kf_test = kf_test.reset_index(drop=True)
    
    kf_train_mols = MOL_class_dict_from_SMILES(kf_train)
    kf_test_mols = MOL_class_dict_from_SMILES(kf_test)

    train_kf = make_all_graphs(kf_train_mols)
    test_kf = make_all_graphs(kf_test_mols)

    kf_train_loader = DataLoader(train_kf, batch_size=len(train_kf), shuffle=True, drop_last=True)
    kf_test_loader = DataLoader(test_kf, batch_size=len(test_kf), shuffle=False, drop_last=True)

    checkpoint = torch.load('model.pt')
    kf_model = Net(args)
    kf_model.load_state_dict(checkpoint['State_dict'])
    kf_model = kf_model.to(device)

    kf_optimizer = optim.Adam(model.parameters(), lr=args.lr)
    kf_optimizer.load_state_dict(checkpoint['optimizer'])

    kf_scheduler = optim.lr_scheduler.StepLR(kf_optimizer, step_size=args.step_size, gamma=args.gamma)
    kf_scheduler.load_state_dict(checkpoint['scheduler'])

    #freezing layers/parameters, play around with this
    count = 0
    for param in kf_model.parameters():
        count += 1
        if count < 15:
            param.requires_grad = False
    
    kf_dict_result = dict()
    args.exp_name = 'kf_test'
    kf_result = vars(kf_experiment(kf_model, kf_optimizer, kf_scheduler, kf_train_loader, kf_test_loader, device, args, trainmodel = True, savemodel = False))
    kf_dict_result[args.exp_name] = copy.deepcopy(kf_result)
    torch.cuda.empty_cache()


    kf_res_df = pd.DataFrame(kf_dict_result).transpose()
    kf_conf = kf_res_df['conf'].iloc[0]
    kf_confusion_matrix = np.array([kf_conf[0], kf_conf[1]], dtype=float)


    CM_list.append(kf_confusion_matrix)

CM_combined = np.zeros((2,2))
for cm in CM_list:
    CM_combined[0,0] += cm[0,0]
    CM_combined[0,1] += cm[0,1]
    CM_combined[1,0] += cm[1,0]
    CM_combined[1,1] += cm[1,1]

kfcm = CM_combined
kf_sensitivity1 = kfcm[1,1]/(kfcm[1,1]+kfcm[1,0])
kf_specificity1 = kfcm[0,0]/(kfcm[0,0]+kfcm[0,1])
kf_er = (kfcm[0,1]+kfcm[1,0])/sum(sum(kfcm))
kf_Sn.append(kf_sensitivity1)
kf_Sp.append(kf_specificity1)
kf_BA.append((kf_sensitivity1 + kf_specificity1)/2)
kf_ER.append(kf_er)

kf_results_df = pd.DataFrame({'GCN_tl_BA':kf_BA, 'GCN_tl_Sn':kf_Sn, 'GCN_tl_Sp':kf_Sp, 'GCN_tl_ER':kf_ER})
kf_results_df.to_csv('GCN_kf_metrics.csv', index=False)
print(kfcm)
display(kf_results_df)

Fold 1 ...


c:\Users\trifo\anaconda3\envs\GCNenv\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Fold 2 ...


c:\Users\trifo\anaconda3\envs\GCNenv\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
c:\Users\trifo\anaconda3\envs\GCNenv\lib\site-packages\sklearn\metrics\_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
c:\Users\trifo\anaconda3\envs\GCNenv\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Fold 3 ...
Fold 4 ...


c:\Users\trifo\anaconda3\envs\GCNenv\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Fold 5 ...


c:\Users\trifo\anaconda3\envs\GCNenv\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
c:\Users\trifo\anaconda3\envs\GCNenv\lib\site-packages\sklearn\metrics\_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
c:\Users\trifo\anaconda3\envs\GCNenv\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Fold 6 ...
[[2. 1.]
 [1. 2.]]


,GCN_tl_BA,GCN_tl_Sn,GCN_tl_Sp,GCN_tl_ER
0,0.666667,0.666667,0.666667,0.333333


In [52]:
#transfer learning through LOO (leave one out) validation for MARINE

#altering experiment function for this portion
def kf_experiment(model, optimizer, scheduler, train_loader, test_loader, device, args, trainmodel = True, savemodel = False): 
    time_start = time.time()
    
    criterion = nn.CrossEntropyLoss()

    #training here
    list_train_loss = list()
    list_train_acc = list()
    if trainmodel:
        #print('[Train]')
        for epoch in range(args.epoch):
            scheduler.step()
            #print('- Epoch :', epoch+1)
            model, train_loss, train_acc = train(model, device, optimizer, train_loader, criterion, args)
            list_train_loss.append(train_loss)
            list_train_acc.append(train_acc)
            #print('Loss : %.4f' % list_train_loss[-1], '| Accuracy : %.4f' % list_train_acc[-1])
    
    #testing here
    conf, total_acc, RB, NRB, miscore, mascore, data_total, pred_data_total, y_pred_list, ba, er = test(model, device, test_loader, args)
    
    time_end = time.time()
    time_required = time_end - time_start
    
    args.list_train_loss = list_train_loss
    args.list_train_acc = list_train_acc
    args.data_total = data_total
    args.pred_data_total = pred_data_total
    args.conf = conf
    args.total_acc = total_acc
    args.RB = RB
    args.NRB = NRB
    args.miscore = miscore
    args.mascore = mascore
    args.time_required = time_required
    args.y_pred_list = y_pred_list
    args.ba = ba
    args.er = er
    
    #saving model state here
    if savemodel:
        save_checkpoint(epoch, model, optimizer, 'model.pt')

    return args

#ai for pfas code, transferlearning notebook, benchmarks bayesian gpyops

####### Model application
args.epoch = 5000
args.lr = 0.0001
args.optim = 'Adam'
args.step_size = 10
args.gamma = 0.9
args.dropout = 0.1
args.n_features = 75
dim = 512
args.conv_dim1 = dim
args.conv_dim2 = dim
args.conv_dim3 = dim
args.concat_dim = dim
args.pred_dim1 = dim
args.pred_dim2 = dim
args.out_dim = 2

kf_df = pd.read_excel('C:/Users/trifo/Argonne SULI Project/src/GCN_RB_NRB_Model/TL_Seawater(Marine)_Dataset.xlsx')
kf_df = pd.concat([kf_df['SMILES'], kf_df['Class']], axis=1)
kfold = KFold(n_splits=len(kf_df))

kf_BA = []
kf_Sn = []
kf_Sp = []
kf_ER = []

CM_list = []

validation = []

for fold, (train_index, test_index) in enumerate(kfold.split(kf_df)):
    print('Fold', fold+1, '...')
    kf_train = kf_df.iloc[train_index]
    kf_test = kf_df.iloc[test_index]
    kf_train = kf_train.reset_index(drop=True)
    kf_test = kf_test.reset_index(drop=True)
    
    kf_train_mols = MOL_class_dict_from_SMILES(kf_train)
    kf_test_mols = MOL_class_dict_from_SMILES(kf_test)

    train_kf = make_all_graphs(kf_train_mols)
    test_kf = make_all_graphs(kf_test_mols)

    kf_train_loader = DataLoader(train_kf, batch_size=len(train_kf), shuffle=True, drop_last=True)
    kf_test_loader = DataLoader(test_kf, batch_size=len(test_kf), shuffle=False, drop_last=True)

    checkpoint = torch.load('model.pt')
    kf_model = Net(args)
    kf_model.load_state_dict(checkpoint['State_dict'])
    kf_model = kf_model.to(device)

    kf_optimizer = optim.Adam(model.parameters(), lr=args.lr)
    kf_optimizer.load_state_dict(checkpoint['optimizer'])

    kf_scheduler = optim.lr_scheduler.StepLR(kf_optimizer, step_size=args.step_size, gamma=args.gamma)
    kf_scheduler.load_state_dict(checkpoint['scheduler'])

    #freezing layers/parameters, play around with this
    count = 0
    for param in kf_model.parameters():
        count += 1
        if count < 23:
            param.requires_grad = False
    
    kf_dict_result = dict()
    args.exp_name = 'kf_test'
    kf_result = vars(kf_experiment(kf_model, kf_optimizer, kf_scheduler, kf_train_loader, kf_test_loader, device, args, trainmodel = True, savemodel = False))
    kf_dict_result[args.exp_name] = copy.deepcopy(kf_result)
    torch.cuda.empty_cache()


    kf_res_df = pd.DataFrame(kf_dict_result).transpose()
    kf_conf = kf_res_df['conf'].iloc[0]
    kf_confusion_matrix = np.array([kf_conf[0], kf_conf[1]], dtype=float)


    CM_list.append(kf_confusion_matrix)

CM_combined = np.zeros((2,2))
for cm in CM_list:
    CM_combined[0,0] += cm[0,0]
    CM_combined[0,1] += cm[0,1]
    CM_combined[1,0] += cm[1,0]
    CM_combined[1,1] += cm[1,1]

kfcm = CM_combined
kf_sensitivity1 = kfcm[1,1]/(kfcm[1,1]+kfcm[1,0])
kf_specificity1 = kfcm[0,0]/(kfcm[0,0]+kfcm[0,1])
kf_er = (kfcm[0,1]+kfcm[1,0])/sum(sum(kfcm))
kf_Sn.append(kf_sensitivity1)
kf_Sp.append(kf_specificity1)
kf_BA.append((kf_sensitivity1 + kf_specificity1)/2)
kf_ER.append(kf_er)

kf_results_df = pd.DataFrame({'GCN_tl_BA':kf_BA, 'GCN_tl_Sn':kf_Sn, 'GCN_tl_Sp':kf_Sp, 'GCN_tl_ER':kf_ER})
kf_results_df.to_csv('GCN_kf_metrics.csv', index=False)
print(kfcm)
display(kf_results_df)

Fold 1 ...


c:\Users\trifo\anaconda3\envs\GCNenv\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Fold 2 ...


c:\Users\trifo\anaconda3\envs\GCNenv\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
c:\Users\trifo\anaconda3\envs\GCNenv\lib\site-packages\sklearn\metrics\_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
c:\Users\trifo\anaconda3\envs\GCNenv\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Fold 3 ...


...

c:\Users\trifo\anaconda3\envs\GCNenv\lib\site-packages\sklearn\metrics\_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
c:\Users\trifo\anaconda3\envs\GCNenv\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Fold 14 ...
Fold 15 ...


c:\Users\trifo\anaconda3\envs\GCNenv\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


[[5. 4.]
 [5. 1.]]


,GCN_tl_BA,GCN_tl_Sn,GCN_tl_Sp,GCN_tl_ER
0,0.361111,0.166667,0.555556,0.6
